# Capstone Project

## Study of Restaurants in Mumbai,India and Opening a new Restaurant

## Import Libraries

In [58]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium 

Solving environment: done

# All requested packages already installed.



In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


In [35]:
from geopy.geocoders import Nominatim

In [38]:
from geopy.geocoders import geocoder as geo

ImportError: cannot import name 'geocoder'

In [43]:
!pip install geocoder

     |████████████████████████████████| 102kB 11.8MB/s ta 0:00:01


In [44]:
import geocoder

## Scrap data from website

In [24]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_of_Mumbai").text

In [25]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [26]:
# create a list to store neighborhood data
neighborhoodList = []

In [27]:
# append the data into the list
for row in soup.find_all("div",class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [28]:
# create a new DataFrame from the list
df = pd.DataFrame({"Neighborhood": neighborhoodList})

df.head(10)

,Neighborhood
0,Andheri
1,Anushakti Nagar
2,Baiganwadi
3,Bandra
4,Bhandup
5,Borivali
6,Charkop
7,Chembur
8,Dahisar
9,Devipada


In [30]:
df.shape

(40, 1)

## Get Coordinates

In [31]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Mumbai, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [45]:
# call the function to get coordinates,store in a new list
coordinates = [ get_latlng(neighborhood) for neighborhood in df["Neighborhood"].tolist() ]

In [49]:
len(coordinates)

40

In [51]:
# create df to store coordinates
df_coordinates = pd.DataFrame(coordinates, columns=['Latitude', 'Longitude'])

In [53]:
#merge both dataframes
df['Latitude'] = df_coordinates['Latitude']
df['Longitude'] = df_coordinates['Longitude']

In [54]:
df.shape

(40, 3)

In [55]:
df.head()

,Neighborhood,Latitude,Longitude
0,Andheri,19.118483,72.841774
1,Anushakti Nagar,19.042830,72.927340
2,Baiganwadi,19.062930,72.926660
3,Bandra,19.054220,72.840190
4,Bhandup,19.145560,72.948560


# Creating Map

In [56]:
#Get Coordinates
address = 'Mumbai, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('coordinates of Mumbai {}, {}.'.format(latitude, longitude))

coordinates of Mumbai 18.9387711, 72.8353355.


In [68]:
# create map of Mumbai
map_m = folium.Map(location=[latitude, longitude], zoom_start=9)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_m)  

In [69]:
map_m

#  Using Foresquare API

In [125]:
CLIENT_ID = 'bgngngf' # your Foursquare ID
CLIENT_SECRET = 'sfgh' # your Foursquare Secret
VERSION = '20190914' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: bgngngf
CLIENT_SECRET:sfgh


### Get Top 100 venues within 2500 meter radius

In [71]:
radius = 2500
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [72]:
df_venues = pd.DataFrame(venues)

# define the column names
df_venues.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']


In [73]:
df_venues.shape

(2999, 7)

In [74]:
df_venues.head()

,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Andheri,19.118483,72.841774,Merwans Cake shop,19.119300,72.845418,Bakery
1,Andheri,19.118483,72.841774,Radha Krishna Veg Restaurant,19.115130,72.843060,Indian Restaurant
2,Andheri,19.118483,72.841774,Naturals,19.111204,72.837255,Ice Cream Shop
3,Andheri,19.118483,72.841774,Shawarma Factory,19.124591,72.840398,Falafel Restaurant
4,Andheri,19.118483,72.841774,Starbucks Coffee : A Tata Alliance,19.114569,72.836205,Coffee Shop


In [76]:
#count values of venues in each Neighbourhood
df_venues.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Andheri,100,100,100,100,100,100
Anushakti Nagar,21,21,21,21,21,21
Baiganwadi,4,4,4,4,4,4
Bandra,100,100,100,100,100,100
Bhandup,38,38,38,38,38,38
Borivali,100,100,100,100,100,100
Charkop,66,66,66,66,66,66
Chembur,100,100,100,100,100,100
Dahisar,83,83,83,83,83,83


In [78]:
len(df_venues['VenueCategory'].unique())

181

In [79]:
df_venues['VenueCategory'].unique()

array(['Bakery', 'Indian Restaurant', 'Ice Cream Shop',
       'Falafel Restaurant', 'Coffee Shop', 'Sandwich Place', 'Pub',
       'Pizza Place', 'Fast Food Restaurant', 'Multiplex', 'Juice Bar',
       'Café', 'Breakfast Spot', 'Theater', 'Snack Place',
       'Seafood Restaurant', 'American Restaurant', 'Cocktail Bar',
       'Chinese Restaurant', 'Beach', 'Bar', 'Food Truck', 'Brewery',
       'Gym / Fitness Center', 'Hotel', 'Mughlai Restaurant',
       'Mediterranean Restaurant', 'Electronics Store', 'BBQ Joint',
       'Diner', 'Italian Restaurant', 'Lounge', 'Asian Restaurant', 'Spa',
       'Department Store', 'Comfort Food Restaurant',
       'Vegetarian / Vegan Restaurant', 'Jewelry Store', "Women's Store",
       'Food', 'Plaza', 'Supermarket', 'Sports Bar', 'Park', 'Platform',
       'Paper / Office Supplies Store', 'Food & Drink Shop',
       'Concert Hall', 'Restaurant', 'Garden', 'Shop & Service',
       'Bus Station', 'Dessert Shop', 'Sports Club', 'Gourmet Shop',
    

# One Hot Encoding

In [82]:
# one hot encoding
df_onehot = pd.get_dummies(df_venues[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
df_onehot['Neighborhoods'] = df_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [df_onehot.columns[-1]] + list(df_onehot.columns[:-1])
df_onehot = df_onehot[fixed_columns]


In [83]:
df_onehot.shape

(2999, 182)

In [84]:
df_onehot.head()

,Neighborhoods,Afghan Restaurant,Airport,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bar,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Boat or Ferry,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Gym,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General Entertainment,German Restaurant,Gift Shop,Goan Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Historic Site,History Museum,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Light Rail Station,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mountain,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Toll Booth,Toy / Game Store,Track,Trail,Train,Train Station,Vegetarian / Vegan Restaurant,Whisky Bar,Wine Bar,Women's Store
0,Andheri,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

## Group Neighbourhood by occurance of each category

In [85]:
df_grouped = df_onehot.groupby(["Neighborhoods"]).mean().reset_index()

In [86]:
df_grouped.shape

(40, 182)

In [87]:
df_grouped

,Neighborhoods,Afghan Restaurant,Airport,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bar,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Boat or Ferry,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Gym,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General Entertainment,German Restaurant,Gift Shop,Goan Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Historic Site,History Museum,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Light Rail Station,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mountain,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Toll Booth,Toy / Game Store,Track,Trail,Train,Train Station,Vegetarian / Vegan Restaurant,Whisky Bar,Wine Bar,Women's Store
0,Andheri,0.000000,0.00,0.020000,0.000000,0.000000,0.000000,0.010000,0.00,0.020000,0.00,0.010000,0.030000,0.020000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.010000,0.010000,0.00,0.000000,0.000,0.000000,0.00,0.100000,0.00,0.060000,0.000000,0.020000,0.010000,0.00,0.00,0.00,0.01,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.010000,0.00,0.000000,0.00,0.010000,0.000000,0.010000,0.000000,0.000000,0.01,0.00,0.000000,0.010000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.00,0.00,0.030000,0.000000,0.050000,0.150000,0.000000,0.00,0.010000,0.00,0.01,0.040000,0.00,0.000000,0.000000,0.030000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.0,0.01,0.000000,0.030000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.020000,0.00,0.000000,0.000000,0.000000,0.000000,0.060000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.030000,0.000000,0.00,0.040000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.00,0.000000,0.01,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.010000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.010000,0.00,0.00,0.010000
1,Anushakti Nagar,0.000000,0.00

In [95]:
len(df_grouped[df_grouped["Restaurant"] > 0])

30

# Create New Dataframe on Restaurant

In [97]:
df_res = df_grouped[["Neighborhoods","Restaurant"]]

In [98]:
df_res.head()

,Neighborhoods,Restaurant
0,Andheri,0.000000
1,Anushakti Nagar,0.047619
2,Baiganwadi,0.000000
3,Bandra,0.010000
4,Bhandup,0.052632


# Using K-Means Clustering

In [101]:
# set number of clusters
kclusters = 5

df_clustering = df_res.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 3, 4, 0, 3, 1, 3, 2, 2, 2], dtype=int32)

In [103]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
df_merged = df_res.copy()

# add clustering labels
df_merged["Cluster Labels"] = kmeans.labels_

In [104]:
df_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)

In [105]:
df_merged

,Neighborhood,Restaurant,Cluster Labels
0,Andheri,0.000000,4
1,Anushakti Nagar,0.047619,3
2,Baiganwadi,0.000000,4
3,Bandra,0.010000,0
4,Bhandup,0.052632,3
5,Borivali,0.070000,1
6,Charkop,0.045455,3
7,Chembur,0.030000,2
8,Dahisar,0.036145,2
9,Devipada,0.040000,2


### Add Latitude And Longitude Data

In [106]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
df_merged = df_merged.join(df.set_index("Neighborhood"), on="Neighborhood")


In [107]:
df_merged.shape

(40, 5)

In [108]:
df_merged

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
0,Andheri,0.000000,4,19.118483,72.841774
1,Anushakti Nagar,0.047619,3,19.042830,72.927340
2,Baiganwadi,0.000000,4,19.062930,72.926660
3,Bandra,0.010000,0,19.054220,72.840190
4,Bhandup,0.052632,3,19.145560,72.948560
5,Borivali,0.070000,1,19.229360,72.857510
6,Charkop,0.045455,3,19.208660,72.826120
7,Chembur,0.030000,2,19.062200,72.902420
8,Dahisar,0.036145,2,19.250030,72.859080
9,Devipada,0.040000,2,19.224720,72.866060


In [110]:
# sort the results by Cluster Labels
df_merged.sort_values(["Cluster Labels"], inplace=True)
df_merged

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
30,Seven Bungalows,0.010000,0,19.128560,72.820850
32,"Sion, Mumbai",0.010000,0,19.046460,72.863130
37,Wadala,0.011494,0,19.017160,72.858130
3,Bandra,0.010000,0,19.054220,72.840190
27,Mulund,0.010000,0,19.171850,72.955640
20,Kausa,0.010000,0,19.127560,72.825400
16,Juhu,0.020000,0,19.014930,72.845220
25,Mira Road,0.020000,0,19.074161,72.861670
24,"Matharpacady, Mumbai",0.010000,0,19.044920,72.867205
21,Kurla,0.020000,0,19.064940,72.880730


# Visualize the Clusters

In [116]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=9)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Neighborhood'], df_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.6).add_to(map_clusters)

In [117]:
map_clusters

# Examine The Clusters

## Cluster 0

In [118]:
df_merged.loc[df_merged['Cluster Labels'] == 0]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
30,Seven Bungalows,0.010000,0,19.128560,72.820850
32,"Sion, Mumbai",0.010000,0,19.046460,72.863130
37,Wadala,0.011494,0,19.017160,72.858130
3,Bandra,0.010000,0,19.054220,72.840190
27,Mulund,0.010000,0,19.171850,72.955640
20,Kausa,0.010000,0,19.127560,72.825400
16,Juhu,0.020000,0,19.014930,72.845220
25,Mira Road,0.020000,0,19.074161,72.861670
24,"Matharpacady, Mumbai",0.010000,0,19.044920,72.867205
21,Kurla,0.020000,0,19.064940,72.880730


## Cluster 1

In [119]:
df_merged.loc[df_merged['Cluster Labels'] == 1]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
29,Pestom sagar,0.06,1,19.070630,72.902200
35,Vashi,0.06,1,19.084700,72.904840
22,Mahavir Nagar (Kandivali),0.07,1,19.211952,72.837542
18,Kandivali,0.07,1,19.211952,72.837542
5,Borivali,0.07,1,19.229360,72.857510
12,Ghatkopar,0.06,1,19.086477,72.908956


## Cluster 2

In [120]:
df_merged.loc[df_merged['Cluster Labels'] == 2]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
33,Thakur village,0.030303,2,19.210200,72.875410
34,Tilak Nagar (Mumbai),0.032258,2,18.996160,72.852810
39,Worli,0.030000,2,19.007450,72.816910
9,Devipada,0.040000,2,19.224720,72.866060
8,Dahisar,0.036145,2,19.250030,72.859080
7,Chembur,0.030000,2,19.062200,72.902420
17,Kalyan,0.040000,2,18.953937,72.820367


## Cluster 3

In [121]:
df_merged.loc[df_merged['Cluster Labels'] == 3]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
14,Grant Road,0.050000,3,18.95929,72.83108
10,Dombivli,0.050000,3,19.21287,73.08324
6,Charkop,0.045455,3,19.20866,72.82612
4,Bhandup,0.052632,3,19.14556,72.94856
1,Anushakti Nagar,0.047619,3,19.04283,72.92734


## Cluster 4

In [122]:
df_merged.loc[df_merged['Cluster Labels'] == 4]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
36,Vikhroli,0.0,4,19.111090,72.927810
0,Andheri,0.0,4,19.118483,72.841774
28,Mumbra,0.0,4,19.190560,73.022670
26,Mogra Village,0.0,4,26.135300,73.054840
23,Mankhurd,0.0,4,19.048530,72.932200
38,Western Suburbs (Mumbai),0.0,4,19.197000,72.827630
15,Jogeshwari,0.0,4,19.137900,72.849410
2,Baiganwadi,0.0,4,19.062930,72.926660
31,Shil Phata,0.0,4,19.146580,73.040050
19,Kanjurmarg,0.0,4,19.131400,72.935650


# Observations

Cluster 0 and cluster 4 have the most number of restaurants while clusters 1,2 and 3 have moderate no of clusters.Restaurants in cluster 0 and 4 have the most competitionwhile restaurants in clusters 1,2 and 3 have moderate competition.So this represents a great opportunity for a person to open a restaurant in these areas.While Clusters 0 and 4 have a oversupply of restaurants they may have adequate amount of customers due to that area being well developed.But the other areas are also well developed and have a undersupply of restaurants.So it is a great opportunuty to open a restaurant there.